# RevIN
- Reversible Instance Normalization
- 시계열 데이터에서 분포 변화(distribution shift) 문제를 해결하기 위해 제안된 정규화 기법
- 시계열 데이터는 센서, 사용자, 환경에 따라 분포가 크게 달라지는데, 이런 분포 변화는 성능 저하의 원인
- RevIN은 데이터를 표준화했다가 모델 출력 후 역변환해서 원래 분포를 복원함
- 표준화하면 모든 샘플을 동일한 기준에서 비교 가능, 복원해서 결과를 실제 단위로 해석 가능
- RevIN은 선택적으로 affine transform (스케일링, 시프트)를 학습할 수 있음 (최적화된 정규화 스케일 학습)
- Subtract Last 옵션: 마지막 시점을 빼서 normalization
- 마지막 값을 빼는 이유?: 마지막 관측값을 기준으로 상대적 변화를 보기 위함




In [ ]:
import torch
import torch.nn as nn

1. `__init__`
- affine이 True면 선형 변환(스케일, 시프트)을 학습
- _init_params(): 학습 가능한 weight, bias를 생성
- subtract_last면 마지막 시점 값을 평균 대신 사용

2. `forward`



In [ ]:
class RevIN(nn.Module):
    def __init__(self, num_features: int, eps=1e-5, affine=True, subtract_last=False):
        """
        :param num_features: the number of features or channels
        :param eps: a value added for numerical stability
        :param affine: if True, RevIN has learnable affine parameters
        """
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.subtract_last = subtract_last
        if self.affine:
            self._init_params()

    def forward(self, x, mode:str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else: raise NotImplementedError
        return x

    def _init_params(self):
        # initialize RevIN params: (C,)
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        dim2reduce = tuple(range(1, x.ndim-1))
        if self.subtract_last:
            self.last = x[:,-1,:].unsqueeze(1)
        else:
            self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        if self.subtract_last:
            x = x - self.last
        else:
            x = x - self.mean
        x = x / self.stdev
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = x - self.affine_bias
            x = x / (self.affine_weight + self.eps*self.eps)
        x = x * self.stdev
        if self.subtract_last:
            x = x + self.last
        else:
            x = x + self.mean
        return x